# Evaluating customer booking trends to identify targeted marketing opportunities



**Overview**

Part 1  Introduction<br>

Part 2  Data Preparation & Data Cleaning<br>

Part 3  Exploratory Data Analysis<br><br>

## Part 1  Introduction

Our project involves the analysis of a customer dataset of a four-star hotel in Lisbon, Portugal. <br>
The dataset comprises of customer behavioral data for the year 2015 to 2018. In addition to personal and behavioral information, the dataset also contains demographic information such as age and geographical information such as customer nationality. In total, there are 31 fields describing a total of 83,590 customers.

Our stakeholder: the marketing team of the hotel

**The objective of our project is to**

1. Gauge customer preferences to devise marketing strategies.

2. Conduct analysis to understand customer segmentation.<br>

**Source**:

Antonio, Nuno; de Almeida, Ana; Nunes, Luis (2020), “Lisbon, Portugal, hotel’s customer dataset with three years of personal, behavioral, demographic, and geographic information”, Mendeley Data, V1, doi: 10.17632/j83f5fsh6c.1
https://data.mendeley.com/datasets/j83f5fsh6c/1

(Kaggle link: https://www.kaggle.com/datasets/nantonio/a-hotels-customers-dataset)<br><br>

## Part 2  Data Preparation & Data Cleaning

#### Check the original data

In [46]:
%%bigquery
SELECT *
FROM `ba775-a09-fall22.hotelcustomerdataset.hotel-customer-dataset`;

Downloading: 100%|██████████| 83590/83590 [00:04<00:00, 17725.26rows/s]


,ID,Nationality,Age,DaysSinceCreation,NameHash,DocIDHash,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,...,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
0,25208,ITA,47,640,1.189864e+76,5.158616e+76,21,246.4,14.0,0,...,0,0,0,0,0,0,0,0,0,0
1,52953,ESP,31,255,9.360460e+76,8.738363e+76,7,159.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
2,52971,CHE,59,255,8.846941e+76,7.281996e+75,2,275.2,30.0,0,...,0,0,0,0,1,0,0,0,0,0
3,52977,DNK,67,255,8.900703e+76,1.122777e+77,35,87.2,1.0,0,...,0,0,0,0,1,0,0,0,0,0
4,52985,ITA,41,255,4.005763e+76,1.062045e+77,85,122.1,7.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83585,83427,CHE,67,1,6.046438e+76,6.422017e+76,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
83586,83572,CHN,34,0,9.163866e+76,5.905219e+76,0,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0
83587,83574,FRA,36,0,5.313667e+76,3.315645e+76,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
83588,83576,FRA,9,0,7.807027e+76,1.045344e+77,0,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


#### Transform data type and drop unnecessary columns
Transform "Age" data type into integer and drop the unnecessary columns including "NameHash" and "DocIDHash".<br>
Save the table as a new table `hotelcustomerdataset.hotelcustomersdatasetcleaned`.

In [2]:
%%bigquery
CREATE TABLE IF NOT EXISTS ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned
AS
select 
ID,Nationality,safe_cast(Age as INT64) Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,
BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,
SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
FROM `ba775-a09-fall22.hotelcustomerdataset.hotel-customer-dataset`;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 526.53query/s] 


""


#### Check if there are any duplicates in "ID"

In [48]:
%%bigquery
--check duplicate ID
SELECT ID,COUNT(*)
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned`
GROUP BY ID
HAVING COUNT(*) > 1;

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 1048.05query/s]
Downloading: 0rows [00:00, ?rows/s]


,ID,f0_


#### Check if there are any null values in "Age"

In [50]:
%%bigquery
select count(Age) Age_Count, count(*) Total, count(*)-count(Age) Null_Count
from `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned`;

Downloading: 100%|██████████| 1/1 [00:04<00:00,  4.78s/rows]


,Age_Count,Total,Null_Count
0,79811,83590,3779


There are 3779 rows have null value in "Age".

#### Identify outliers of "Age" with the lower/upper bound equals to 2 standard deviation below/above the mean

In [52]:
%%bigquery
SELECT
   data.Age,
   case
      when data.Age between mean - 2 * stdev and mean + 2 * stdev 
         then 'not outlier'
      else 'outlier'
   end label,
   mean - 2 * stdev lower_bound,
   mean + 2 * stdev upper_bound
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned` AS data
CROSS JOIN 
   (
    SELECT
      avg(data.Age) mean,
      stddev_samp(data.Age) stdev 
    FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned` AS data ) mean_sd
ORDER BY Age DESC
limit 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.63rows/s]


,Age,label,lower_bound,upper_bound
0,122,outlier,12.253291,78.542764
1,114,outlier,12.253291,78.542764
2,114,outlier,12.253291,78.542764
3,113,outlier,12.253291,78.542764
4,113,outlier,12.253291,78.542764


#### Count the numbers of outliers in "Age"

In [53]:
%%bigquery
select label,count(label)counts
from (SELECT
   data.Age,
   case
      when data.Age between mean - 2 * stdev and mean + 2 * stdev 
         then 'not outlier'
      else 'outlier'
   end label,
   mean - 2 * stdev lower_bound,
   mean + 2 * stdev upper_bound
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned` AS data
CROSS JOIN 
   (
    SELECT
      avg(data.Age) mean,
      stddev_samp(data.Age) stdev 
    FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned` AS data ) mean_sd
ORDER BY label DESC)
group by label;

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.14rows/s]


,label,counts
0,not outlier,75944
1,outlier,7646


#### Create a new table 
Exclude outliers in "Age", "DaysSinceLastStay" and "DaysSinceFirstStay", to drop null values and unusual values in "Age" and customers who have never stayed in this hotel before("DaysSinceLastStay"= -1, "DaysSinceFirstStay"= -1).<br>
Add new columns of "AgeRange", "Country"(full name of Nationality) and "TotalRevenue"(sum of "LodgingRevenue" and "OtherRevenue").<br>
The new table `hotelcustomerdataset.hotelcustomersdatasetcleaned_2` will be used in the following parts of our analysis.

In [3]:
%%bigquery
CREATE or replace Table `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
as 
select 
A.ID,
Nationality,
C.Country,
Age,
cast(B.AgeRange as STRING) AgeRange,
DaysSinceCreation,
AverageLeadTime,
LodgingRevenue,
OtherRevenue,
LodgingRevenue+OtherRevenue as TotalRevenue,
BookingsCanceled,
BookingsNoShowed,
BookingsCheckedIn,
PersonsNights,
RoomNights,
DaysSinceLastStay,
DaysSinceFirstStay,
DistributionChannel,
MarketSegment,
SRHighFloor,
SRLowFloor,
SRAccessibleRoom,
SRMediumFloor,
SRBathtub,
SRShower,
SRCrib,
SRKingSizeBed,
SRTwinBed,
SRNearElevator,
SRAwayFromElevator,
SRNoAlcoholInMiniBar,
SRQuietRoom
from ( (select *
from `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned`
where Age is not null
and Age > 12
and Age < 79
and DaysSinceLastStay != -1
and DaysSinceFirstStay != -1) A
left join
(SELECT ID, 
Case when Age > 10 and Age < 20 then '10s'
     when Age >= 20 and Age < 30 then '20s'
     when Age >= 30 and Age < 40 then '30s'
     when Age >= 40 and Age < 50 then '40s'
     when Age >= 50 and Age < 60 then '50s'
     when Age >= 60 and Age < 70 then '60s'
     Else '70s' END as AgeRange
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned`)B
on A.ID=B.ID)
left join
(
 SELECT country as Country, country_code
 FROM `bigquery-public-data.world_bank_global_population.population_by_country`
) C
ON A.Nationality=C.country_code;

Query complete after 0.00s: 100%|██████████| 6/6 [00:00<00:00, 2074.33query/s]                        


""


## Part 3  Exploratory Data Analysis

Our EDA focus on the following features: 

**Demographic Information**: 
- Age
- Nationality


**Revenue Generated**: 

- Revenue generated from price of room


**Booking Information**: 

- Booking distribution channel 
- Booking lead time (booking time ahead of arrival) 
- Repeated booking from same customer 
- Special requests 

### Demographic Information -- Age 

**Which age group should be targeted to maximize total revenue?**

In [59]:
%%bigquery
SELECT AgeRange,
    SUM(TotalRevenue) AS total_revenue,
    ROUND(SUM(TotalRevenue)/
    (SELECT SUM(TotalRevenue) FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2)*100,2) AS Percentage_Of_Total_Revenue,
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY AgeRange
ORDER BY total_revenue DESC;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  5.73rows/s]


,AgeRange,total_revenue,Percentage_Of_Total_Revenue
0,50s,7253303.44,25.11
1,40s,7005467.78,24.26
2,30s,5269102.34,18.24
3,60s,3850725.98,13.33
4,20s,3038191.01,10.52
5,70s,1665373.38,5.77
6,10s,800131.74,2.77


Approximately 50% of hotel's total revenue comes from customers in their 40s or 50s. <br>

**What is revenue per booking generated by customers from different age groups?**

In [140]:
%%bigquery
SELECT AgeRange,
    SUM(TotalRevenue) AS total_revenue,
    SUM(BookingsCheckedIn) AS Number_of_bookings,
    ROUND(SUM(TotalRevenue)/ SUM(BookingsCheckedIn),0) AS Revenue_per_booking
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY AgeRange;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.76rows/s]


,AgeRange,total_revenue,Number_of_bookings,Revenue_per_booking
0,60s,3850725.98,8437,456.0
1,50s,7253303.44,15088,481.0
2,20s,3038191.01,6412,474.0
3,40s,7005467.78,14915,470.0
4,30s,5269102.34,12163,433.0
5,70s,1665373.38,3753,444.0
6,10s,800131.74,1092,733.0


The revenue per booking generated across different age groups is surprisingly higher in the range of 10s. A possible explanation for this is that customers travelling with families are more likely to book bigger / expensive rooms, as well as spend more on utilities such as food.

### Demographic Information -- Nationality

**Customers from which country have the highest contribution to total revenue?**

In [57]:
%%bigquery
SELECT Country, 
    ROUND(SUM(TotalRevenue),0) AS total_revenue, 
    ROUND(SUM(TotalRevenue)/
    (SELECT SUM(TotalRevenue) FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2)*100,2) AS `Percentage_Of_Total_Revenue`,
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY Country
ORDER BY total_revenue DESC
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.65rows/s]


,Country,total_revenue,Percentage_Of_Total_Revenue
0,France,4767023.0,16.51
1,Germany,3692041.0,12.78
2,United Kingdom,3192975.0,11.06
3,Portugal,2252467.0,7.80
4,Spain,1768904.0,6.12


16.51% of the hotel's revenue comes from customers based in France, followed by Germany, and UK. So the marketing strategies should be focussed on travellers from these locations.

**Travellers from which country contribute to the highest revenue per booking?**

In [58]:
%%bigquery
SELECT Country, 
    ROUND(SUM(TotalRevenue),0) AS total_revenue, 
    SUM(BookingsCheckedIn) AS Number_of_bookings,
    ROUND(SUM(TotalRevenue)/ SUM(BookingsCheckedIn),0) AS Revenue_per_booking
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY Country
ORDER BY total_revenue DESC
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.65rows/s]


,Country,total_revenue,Number_of_bookings,Revenue_per_booking
0,France,4767023.0,9389,508.0
1,Germany,3692041.0,7567,488.0
2,United Kingdom,3192975.0,6524,489.0
3,Portugal,2252467.0,7923,284.0
4,Spain,1768904.0,4022,440.0


Following the same trend as that of total revenue, customers from France have generated the highest revenue per booking. <br><br>

### Revenue Generated

**What is the average price paid per room (in euros)?**

In [72]:
%%bigquery
SELECT SUM(TotalRevenue)/SUM(RoomNights) AS Avg_price_per_room
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2; 

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.23s/rows]


,Avg_price_per_room
0,155.648524


The average price paid per room across all travellers for the years 2015-2018 is 155.64 euros. 

**Travelers from which country tend to pay the highest average price per room?**

In [136]:
%%bigquery
SELECT ROUND(AVG(TotalRevenue/RoomNights),2) AS Avg_PricePerRoom, Country, COUNT(ID) AS Number_of_Bookings
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2 
GROUP BY Country
ORDER BY Avg_PricePerRoom DESC
LIMIT 10;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.02rows/s]


,Avg_PricePerRoom,Country,Number_of_Bookings
0,478.94,Namibia,6
1,462.00,Sao Tome and Principe,1
2,401.81,Mali,4
3,369.00,Zimbabwe,1
4,349.38,Andorra,2
5,336.80,Gibraltar,2
6,324.14,Cuba,7
7,279.22,"Congo, Dem. Rep.",2
8,274.75,Cayman Islands,1
9,269.75,Faroe Islands,1


Nambia, with an average price of 478.94 euros. However, it is important to note that the number of bookings from these countries is significantly low. Thus, to increase revenue, marketing strategies must be aimed at increasing travellers from these 10 countries. 

**Among the top 10 countries that contribute most to the hotel's bookings, travellers from which country pay the highest average price per room?**

In [137]:
%%bigquery
SELECT ROUND(AVG(TotalRevenue/RoomNights),2) AS Avg_PricePerRoom, Country
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2 
GROUP BY Country
HAVING COUNT(ID)>=1969
ORDER BY Avg_PricePerRoom DESC;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.38rows/s]


,Avg_PricePerRoom,Country
0,171.75,Netherlands
1,171.59,Belgium
2,167.97,United States
3,164.69,Spain
4,163.24,United Kingdom
5,159.25,Italy
6,157.16,France
7,153.60,Brazil
8,149.44,Germany
9,143.19,Portugal


Travellers from Netherlands, which has a total of 2037 bookings from the years 2015 to 2018, pay the highest average price per room of 171.75 euros.

### Booking Information -- Booking distribution channel

**Which is the most profitable distribution channel?**

In [61]:
%%bigquery
SELECT DistributionChannel,
       ROUND(SUM(TotalRevenue),0) AS Total_Revenue,
       ROUND(SUM(TotalRevenue)/
       (SELECT SUM(TotalRevenue)
        FROM`ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`)*100,2) AS Percentage
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
GROUP BY DistributionChannel
ORDER BY Total_Revenue DESC;

Downloading: 100%|██████████| 4/4 [00:01<00:00,  2.68rows/s]


,DistributionChannel,Total_Revenue,Percentage
0,Travel Agent/Operator,23693538.0,82.03
1,Direct,4121015.0,14.27
2,Corporate,909255.0,3.15
3,Electronic Distribution,158488.0,0.55


Travel Agent/ Operator is the most profitable distribution channel for the hotel with 82% of the total revenue coming from the given distribution channel.<br>
This is followed by the direct distribution channel, which contributes to 14% of the total revenue. <br>
Corporate and Electronic Distribution channels, combined, contribute merely 3.7% to the total revenue.

**Which distribution channel bring in the highest number of bookings?**

In [62]:
%%bigquery
SELECT DistributionChannel,
       SUM(BookingsCheckedIn) AS Total_Bookings,
       ROUND(SUM(BookingsCheckedIn)/
       (SELECT SUM(BookingsCheckedIn)
             FROM`ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`)*100,2) AS Percentage
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
GROUP BY DistributionChannel
ORDER BY Total_Bookings DESC;

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.07rows/s]


,DistributionChannel,Total_Bookings,Percentage
0,Travel Agent/Operator,49797,80.5
1,Direct,8351,13.5
2,Corporate,3217,5.2
3,Electronic Distribution,495,0.8


80% of the total bookings result from Travel Agents. This is followed by direct distribution channel, which contributes 13.5% to the total number of bookings for the given period. Bookings from corporates count 5% of the total number of bookings.<br>

**Which distribution channel has the highest revenue per booking?**

In [139]:
%%bigquery
SELECT  DistributionChannel,
       SUM(TotalRevenue) AS Total_Revenue,
       SUM(BookingsCheckedIn) AS Total_Bookings,
       ROUND(SUM(TotalRevenue)/SUM(BookingsCheckedIn),2) AS Revenue_per_booking
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
GROUP BY DistributionChannel
ORDER BY Revenue_per_booking DESC;

Downloading: 100%|██████████| 4/4 [00:01<00:00,  2.50rows/s]


,DistributionChannel,Total_Revenue,Total_Bookings,Revenue_per_booking
0,Direct,4121014.89,8351,493.48
1,Travel Agent/Operator,23693538.32,49797,475.80
2,Electronic Distribution,158487.85,495,320.18
3,Corporate,909254.61,3217,282.64


Although the distribution channel 'Travel Agent/ Operator' brings in the highest total revenue and the highest number of bookings, the revenue per booking is higher for the direct distribution channel, indicating that customers via the direct channel either have a preference for high end rooms / services or stay for a longer period of time.<br>

**Which age group do customers booking via Travel Agents belong to?**

In [64]:
%%bigquery
SELECT DistributionChannel,
       AgeRange,
       Count(AgeRange) AS count_per_agegroup,
       ROUND(Count(AgeRange)/
            (SELECT Count(AgeRange) 
             FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
             WHERE DistributionChannel = 'Travel Agent/Operator')*100,2)  AS Percenatge         
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
WHERE DistributionChannel = 'Travel Agent/Operator'
GROUP BY DistributionChannel, AgeRange
ORDER BY count_per_agegroup DESC;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  5.07rows/s]


,DistributionChannel,AgeRange,count_per_agegroup,Percenatge
0,Travel Agent/Operator,50s,11904,24.31
1,Travel Agent/Operator,40s,11046,22.56
2,Travel Agent/Operator,30s,9543,19.49
3,Travel Agent/Operator,60s,7007,14.31
4,Travel Agent/Operator,20s,5334,10.89
5,Travel Agent/Operator,70s,3260,6.66
6,Travel Agent/Operator,10s,866,1.77


Majority of the customers (66%) booking via travel agents are aged between 30 to 60 years.

**Which age group do customers booking via the Direct Channel belong to?**

In [66]:
%%bigquery
SELECT DistributionChannel,
       AgeRange,
       Count(AgeRange) AS count_per_agegroup,
       ROUND(Count(AgeRange)/
            (SELECT Count(AgeRange) 
             FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
             WHERE DistributionChannel = 'Direct')*100,2)  AS Percenatge         
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
WHERE DistributionChannel = 'Direct'
GROUP BY DistributionChannel, AgeRange
ORDER BY count_per_agegroup DESC;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  6.74rows/s]


,DistributionChannel,AgeRange,count_per_agegroup,Percenatge
0,Direct,40s,2199,28.01
1,Direct,50s,1817,23.15
2,Direct,30s,1694,21.58
3,Direct,60s,893,11.38
4,Direct,20s,765,9.75
5,Direct,70s,324,4.13
6,Direct,10s,158,2.01


50% of the customers booking directly are within the age group of 40s to 50s. 

### Booking Information -- Booking lead time

**Which age group has the highest or the lowest time lapse between the date of booking and the date of arrival?**

In [71]:
%%bigquery
SELECT AVG(AverageLeadTime) AS AvgLeadTime, AgeRange
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY AgeRange
ORDER BY AvgLeadTime DESC;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  5.80rows/s]


,AvgLeadTime,AgeRange
0,158.772777,70s
1,107.074830,10s
2,104.810754,60s
3,86.935446,50s
4,74.893936,40s
5,70.304928,20s
6,66.786387,30s


Highest- People in their 70”s<br>
Lowest- People in their 30’s

The average lead time, which is the average number of days elapsed between the customer’s booking date and arrival date, decreases with a decrease in age. Thus, it can be concluded that people in their 30's are more likely to make impromptu decisions than people in the age range of 70's. Therefore, marketing strategies must be timed accordingly for different age segments.

### Booking Information -- Repeated booking from same customer

**Customers from which country have stayed more than once and have contributed to the maximum revenue?**

In [129]:
%%bigquery
--Customers from which countries have arrived more than once and have contributed to the maximum revenue
SELECT country, 
  CASE WHEN DaysSinceLastStay = DaysSinceFirstStay THEN 'One-time Customer'
    ELSE 'Repeat Customers'
    END AS Customer_category,
  SUM(BookingsCheckedIn) AS Number_of_bookings,
ROUND(SUM(TotalRevenue),0) AS Revenue_from_repeat_customers,
ROUND(SUM(TotalRevenue)/SUM(BookingsCheckedIn),0) AS Revenue_per_booking
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY country,customer_category
HAVING Customer_category = 'Repeat Customers'
ORDER BY Revenue_from_repeat_customers DESC
LIMIT 5;

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.57rows/s]


,country,Customer_category,Number_of_bookings,Revenue_from_repeat_customers,Revenue_per_booking
0,Portugal,Repeat Customers,1795,299928.0,167.0
1,France,Repeat Customers,220,123570.0,562.0
2,United Kingdom,Repeat Customers,216,104421.0,483.0
3,Germany,Repeat Customers,191,78247.0,410.0
4,United States,Repeat Customers,154,75854.0,493.0


A remarkable portion of the hotel's bookings are generated from recurring customers, coming from Portugal. However, it can be noted that the corespending revenue per booking generated from customers coming from Portugal is significantly lower than other countries. This can be explained by a significant difference in variables such as duration of stay, utilities required, or number of rooms booked when travelling locally versus internationally. 

**Customers from which age group are regulars at the hotel, and have contributed to the maximum revenue?**

In [141]:
%%bigquery
SELECT AgeRange, 
  CASE WHEN DaysSinceLastStay = DaysSinceFirstStay THEN 'One-time Customer'
    ELSE 'Repeat Customer'
    END AS Customer_category,
SUM(BookingsCheckedIn) AS Number_of_bookings,
SUM(TotalRevenue) AS Revenue_from_repeat_customers
FROM ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2
GROUP BY AgeRange,customer_category
HAVING Customer_category = 'Repeat Customer'
ORDER BY Revenue_from_repeat_customers DESC

Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.81rows/s]


,AgeRange,Customer_category,Number_of_bookings,Revenue_from_repeat_customers
0,50s,Repeat Customer,1067,367520.30
1,40s,Repeat Customer,1220,354088.34
2,30s,Repeat Customer,587,208102.18
3,60s,Repeat Customer,509,185321.53
4,70s,Repeat Customer,169,69573.00
5,20s,Repeat Customer,187,65339.00
6,10s,Repeat Customer,70,6122.25


Most of the hotel's repeat customers are in their 40s or 50s. The marketing campaign should focus on retaining these loyal customers via targeted marketing. 

### Booking Information -- Special requests

**What are the total number of requests made for a specific bed size?**

In [79]:
%%bigquery
SELECT sum(SRKingSizeBed) SRKingSizedbed_count, 
       round(sum(SRKingSizeBed)/count(*)*100,2) SRKingSizedbed_ratio, 
       sum(SRTwinBed) SRTwinBed_count,
       round(sum(SRTwinBed)/count(*)*100,2) SRTwinBed_ratio
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2` ;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.28s/rows]


,SRKingSizedbed_count,SRKingSizedbed_ratio,SRTwinBed_count,SRTwinBed_ratio
0,21019,35.41,7942,13.38


About 35% of the customers have made a special request for King sized bed, and about 13% of the customers have made a special request for a Twin-bed Type. In total, nearly half of the customers (49%) have special request for the bed type, with the King sized bed type being more popular than the twin-bed type.<br><br>

**Among the top 10 countries with the highest number of bookings, what percentage of customers have made special requests for king sized bed and twin-bed for that given country?**

In [81]:
%%bigquery
SELECT 
Country, 
sum(SRKingSizeBed)as SRKingSizedbed_count, 
round(sum(SRKingSizeBed)/count(ID)*100,2) SRKingSizedbed_percentage, 
sum(SRTwinBed)as SRTwinBed_count,
round(sum(SRTwinBed)/count(ID)*100,2) SRTwinBed_percentage
FROM `ba775-a09-fall22.hotelcustomerdataset.hotelcustomersdatasetcleaned_2`
GROUP BY Country
having count(ID)>=1969
order by SRKingSizedbed_percentage desc;

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.94rows/s]


,Country,SRKingSizedbed_count,SRKingSizedbed_percentage,SRTwinBed_count,SRTwinBed_percentage
0,Brazil,951,48.30,274,13.92
1,United States,944,40.19,336,14.30
2,Italy,991,39.67,282,11.29
3,France,3553,38.35,967,10.44
4,United Kingdom,2390,37.30,1148,17.92
5,Belgium,796,34.91,356,15.61
6,Netherlands,700,34.36,355,17.43
7,Spain,1241,31.85,468,12.01
8,Portugal,1969,30.15,570,8.73
9,Germany,2065,27.69,1066,14.30


Nearly half of the customers from Brazil(48%) have indicated a special request for King-sized beds. A similar trend can be noted for customers across all countries. Thus it can be inferred that the people making most bookings at the hotel demand a King-sized bed. Therefore, it is an important variable when creating marketing strategies for the hotel.

## References

Markdown for Jupyter Cheat Sheet https://notebook.community/tschinz/iPython_Workspace/00_Admin/CheatSheet/Markdown%20CheatSheet

SQL STDEV Function https://www.tutorialgateway.org/sql-stdev-function/

A hotel's customers personal, behavioral, demographic, and geographic dataset from Lisbon, Portugal (2015–2018) Nuno Antonio, Ana de Almeida, and Luis Nunes for Data in Brief https://www.sciencedirect.com/science/article/pii/S2352340920314645?via%3Dihub
